# dislocation_monopole Calculation

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2018-08-31

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)

## Introduction

The dislocation_monopole calculation calculation inserts a dislocation monopole into an atomic system using the anisotropic elasticity solution for a perfectly straight dislocation. The system is divided into two regions: a boundary region at the system's edges perpendicular to the dislocation line, and an active region around the dislocation. After inserting the dislocation, the atoms within the active region of the system are relaxed while the positions of the atoms in the boundary region are held fixed at the elasticity solution. The relaxed dislocation system and corresponding dislocation-free base systems are retained in the calculation's archived record. Various properties associated with the dislocation's elasticity solution are recorded in the calculation's results record.

__Version notes__: The orientations of the pre-defined dislocation configurations in the dislocation_monopole records have been changed from iprPy versions 0.7.X and earlier. The new definitions are more consistent with creating dislocations for any crystal structure and so that the slip planes correspond to the planes defined in the stacking_fault records.

__Disclaimer #1__: This calculation method holds the boundary atoms fixed during the relaxation process which results in a mismatch stress at the border between the active and fixed regions that interacts with the dislocation.  Increasing the distance between the dislocation and the boundary region, i.e. increasing the system size, will reduce the influence of the mismatch stresses.

__Disclaimer #2__: The boundary atoms are fixed at the elasticity solution, which assumes the dislocation to be compact (not spread out, dissociated into partials) and to remain at the center of the system.  An alternate simulation design or boundary region method should be used if you want accurate simulations of dislocations with wide cores and/or in which the dislocation moves long distances.

__Disclaimer #3__: The calculation allows for the system to be relaxed either using only static energy/force minimizations or with molecular dynamic steps followed by a minimization.  Only performing a static relaxation is considerably faster than performing a dynamic relaxation, but the dynamic relaxation is more likely to result in a better final dislocation structure.

## Method and Theory

### Stroh theory

A detailed description of the Stroh method to solve the Eshelby solution for an anisotropic straight dislocation can be found in the atomman documentation.

### Orientation

One of the most important considerations in defining an atomistic system containing a dislocation monopole system is the system's orientation.  In particular, care is needed to properly align the system's Cartesian axes, $x, y, z$, the system's box vectors, $a, b, c$, and the Stroh solution's axes, $u, m, n$.

- In order for the dislocation to remain perfectly straight in the atomic system, the dislocation line, $u$, must correspond to one of the system's box vectors.  The resulting dislocation monopole system will be periodic along the box direction corresponding to $u$, and non-periodic in the other two box directions.

- The Stroh solution is invariant along the dislocation line direction, $u$, thereby the solution is 2 dimensional. $m$ and $n$ are the unit vectors for the 2D axis system used by the Stroh solution. As such, $u, m$ and $n$ are all normal to each other. The plane defined by the $um$ vectors is the dislocation's slip plane, i.e. $n$ is normal to the slip plane.

- For LAMMPS simulations, the system's box vectors are limited such that $a$ is parallel to the $x$-axis, and $b$ is in the $xy$-plane (i.e. has no $z$ component). Based on this and the previous two points, the most convenient, and therefore the default, orientation for a generic dislocation is to

  - Make $u$ and $a$ parallel, which also places $u$ parallel to the $x$-axis.

  - Select $b$ such that it is within the slip plane. As $u$ and $a$ must also be in the slip plane, the plane itself is defined by $a \times b$.
  
  - Given that neither $a$ nor $b$ have $z$ components, the normal to the slip plane will be perpendicular to $z$.  From this, it naturally follows that $m$ can be taken as parallel to the $y$-axis, and $n$ parallel to the $z$-axis.

### Calculation methodology

1. An initial system is generated based on the loaded system and *uvw*, *atomshift*, and *sizemults* input parameters.  This initial system is saved as base.dump.

2. The atomman.defect.Stroh class is used to obtain the dislocation solution based on the defined $m$ and $n$ vectors, $C_{ij}$, and the dislocation's Burgers vector, $b_i$.

3. The dislocation is inserted into the system by displacing all atoms according to the Stroh solution's displacements.  The dislocation line is placed parallel to the specified *dislocation_lineboxvector* and includes the Cartesian point (0, 0, 0).

4. The box dimension parallel to the dislocation line is kept periodic, and the other two box directions are made non-periodic. A boundary region is defined that is at least *bwidth* thick at the edges of the two non-periodic directions, such that the shape of the active region corresponds to the *bshape* parameter. Atoms in the boundary region are identified by altering their integer atomic types.

5. The dislocation is relaxed by performing a LAMMPS simulation.  The atoms in the active region are allowed to relax either with nvt integration followed by an energy/force minimization, or with just an energy/force minimization.  The atoms in the boundary region are held fixed at the elastic solution.  The resulting relaxed system is saved as disl.dump.

6. Parameters associated with the Stroh solution are saved to the results record.

## Demonstration

### 1. Setup

#### 1.1 Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import glob
import shutil
import random
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np 

# https://matplotlib.org/
import matplotlib.pyplot as plt
%matplotlib inline

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

Notebook last executed on 2018-09-24 using iprPy version 0.8.2


#### 1.2. Default calculation setup

In [2]:
# Specify calculation style
calc_style = 'dislocation_monopole'

# If workingdir is already set, then do nothing (already in correct folder)
try:
    workingdir = workingdir

# Change to workingdir if not already there
except:
    workingdir = os.path.join(os.getcwd(), 'calculationfiles', calc_style)
    if not os.path.isdir(workingdir):
        os.mkdir(workingdir)
    os.chdir(workingdir)

# Default iprPy library directory
librarydir = os.path.join(iprPy.rootdir, '..', 'library')

### 2. Assign values for the calculation's run parameters

#### 2.1. Specify system-specific paths

- __lammps_command__ (required) is the LAMMPS command to use.

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [3]:
lammps_command = 'lmp_mpi'
mpi_command = 'C:/Program Files/MPICH2/bin/mpiexec -localonly 4'

#### 2.2. Load interatomic potential

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for the calculation.  

- __potential_file__ gives the path to the potential_LAMMPS reference record to use.  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential_dir__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file).  Here, this parameter is automatically generated using potential_name and librarydir.

- __potential__ is an atomman.lammps.Potential object (required).  Here, this parameter is automatically generated from potential_file and potential_dir.

In [4]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'

# Define potential_file and potential_dir using librarydir and potential_name
potential_file = os.path.join(librarydir, 'potential_LAMMPS', potential_name) + '.json'
potential_dir = os.path.join(librarydir, 'potential_LAMMPS', potential_name)

# Initialize Potential object using potential_file and potential_dir.
potential = lmp.Potential(potential_file, potential_dir)
print('Successfully loaded potential', potential)

Successfully loaded potential 1999--Mishin-Y--Ni--LAMMPS--ipr1


#### 2.3. Load initial unit cell system

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to load. 

- __symbols__ is a list of the potential's elemental model symbols to associate with the unique atom types of the loaded system. 

- __box_parameters__ is a list of the a, b, c lattice constants to assign to the loaded file.

- __load_file__ gives the path to the atomic configuration file to load for the ucell system.  Here, this is generated automatically using prototype_name and librarydir.

- __load_style__ specifies the format of load_file.  Here, this is automatically set for crystal_prototype records.

- __load_options__ specifies any other keyword options for properly loading the load_file.  Here, this is automatically set for crystal_prototype records.

- __ucell__ is an atomman.System representing a fundamental unit cell of the system (required).  Here, this is generated using the load_* parameters and symbols.

In [5]:
prototype_name = 'A1--Cu--fcc'
symbols = ['Ni']
box_parameters = uc.set_in_units([3.52, 3.52, 3.52], 'angstrom')

# Define load_file using librarydir and prototype_name
load_file = os.path.join(librarydir, 'crystal_prototype', prototype_name+'.json')

# Define load_style and load_options for crystal_prototype records
load_style = 'system_model'
load_options = {}

# Create ucell by loading prototype record
ucell = am.load(load_style, load_file, symbols=symbols, **load_options)

# Rescale ucell using box_parameters
ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)

print(ucell)

avect =  [ 3.520,  0.000,  0.000]
bvect =  [ 0.000,  3.520,  0.000]
cvect =  [ 0.000,  0.000,  3.520]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Ni',)
pbc = [ True  True  True]
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   1.760 |   1.760
      2 |       1 |   1.760 |   0.000 |   1.760
      3 |       1 |   1.760 |   1.760 |   0.000


#### 2.4 Specify material elastic constants

Simple input parameters:

- __C_dict__ is a dictionary containing the unique elastic constants for the potential and crystal structure defined above. 

Derived parameters

- __C__ is an atomman.ElasticConstants object built from C_dict.

In [6]:
C_dict = {}
C_dict['C11'] = uc.set_in_units(247.86, 'GPa')
C_dict['C12'] = uc.set_in_units(147.83, 'GPa')
C_dict['C44'] = uc.set_in_units(124.84, 'GPa')

# -------------- Derived parameters -------------- #
# Build ElasticConstants object from C_dict terms
C = am.ElasticConstants(**C_dict)

#### 2.5 Specify the defect parameters

- __dislocation_name__ gives the name of the dislocation_monopole reference record in the iprPy library to use for potential. 

- __dislocation_file__ gives the path to a dislocation_monopole reference containing defect input parameters. Here, this is built automatically using dislocation_name and librarydir.

- __dislocation_kwargs__ is a dictionary containing parameters for generating the defect. Values are extracted from the dislocation_model record and uniquely define a type of stacking fault. Included keywords are:

    - __burgers__ is the crystallographic Burgers vector for the dislocation
    
    - __a_uvw, b_uvw, c_uvw__ specifies how to orient the system by defining the crystal vectors of ucell to place along the three system box vectors.
    
    - __atomshift__ is a 3D vector rigid-body shift to apply to atoms in the system. The atomshift vector is relative to the rotated unit cell's box vectors.
    
    - __stroh_m__ a Cartesian unit vector used by the Stroh method to define the created dislocation's orientation and type. This vector is in the dislocation's slip plane and perpendicular to the dislocation's line direction. Default value is \[0, 1, 0\], i.e. along the y-axis. 
    
    - __stroh_n__ a Cartesian unit vector used by the Stroh method to define the created dislocation's orientation and type. This vector is normal to the dislocation's slip plane. Default value is \[0, 0, 1\], i.e. along the z-axis.
    
    - __lineboxvector__ the system box vector, 'a', 'b', or 'c', along which the dislocation line is placed parallel to. Default value is 'a'.

In [7]:
#dislocation_name = 'A1--Cu--fcc--a-2-110--0-screw--{111}'
dislocation_name = 'A1--Cu--fcc--a-2-110--90-edge--{111}'

# Define dislocation_fi;e using librarydir and dislocation_name
dislocation_file = os.path.join(librarydir, 'dislocation_monopole', dislocation_name+'.json')

# Create dictionary of input parameters related to the defect
inputs = {'ucell':ucell, 'dislocation_file':dislocation_file}

#### 2.6 Specify calculation-specific run parameters

Simple input parameters:

- __boundarywidth__ sets the minimum width of the fixed-atom boundary region. This is taken in units of the a lattice parameter of the unit cell.

- __boundaryshape__ specifies what shape to make the fixed-atom boundary region.

    - __circle__ will create a cylindrical active region.
    
    - __rect__ will create a rectangular active region.

- __annealtemperature__ is the temperature at which to relax (anneal) the dislocation system. If annealtemperature is 0.0, then only a static relaxation will be performed. Default value is 0.0.

- __randomseed__ allows for the random seed used in generating initial atomic velocities for a dynamic relaxation to be specified. This is an integer between 1 and 900000000. Default value is None, which will randomly pick a number in that range.

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units. 

In [8]:
boundarywidth = 3
boundaryshape = 'circle'

annealtemperature = 50.0
randomseed = None

energytolerance = 0.0
forcetolerance = uc.set_in_units(1e-6, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')

inputs['dislocation_boundaryshape'] = boundaryshape,
inputs['dislocation_boundarywidth'] = boundarywidth

#### 2.7 Process defect input parameters 

Here, the defect parameters are read in and processed.

In [9]:
iprPy.input.interpret('dislocationmonopole', inputs)

# Show inputs
inputs

{'ucell': <atomman.core.System.System at 0x1c672518>,
 'dislocation_file': 'c:\\users\\lmh1\\documents\\python-packages\\iprpy\\iprPy\\..\\library\\dislocation_monopole\\A1--Cu--fcc--a-2-110--90-edge--{111}.json',
 'dislocation_boundaryshape': ('circle',),
 'dislocation_boundarywidth': 3.0,
 'dislocation_family': 'A1--Cu--fcc',
 'dislocation_stroh_m': '0 1 0',
 'dislocation_stroh_n': '0 0 1',
 'dislocation_lineboxvector': 'a',
 'a_uvw': '-1 -1  2',
 'b_uvw': ' 1 -1  0',
 'c_uvw': ' 1  1  1',
 'atomshift': ' 0.00 0.00 0.25',
 'dislocation_burgersvector': ' 0.5 -0.5  0.0',
 'dislocation_model': DataModelDict([('key',
                 'd59f2382-17e5-4fbd-b398-407c75e6009a'),
                ('id', 'A1--Cu--fcc--a-2-110--90-edge--{111}'),
                ('character', 'edge'),
                ('Burgers-vector', 'a/2[ 1,-1, 0]'),
                ('slip-plane', [1, 1, 1]),
                ('line-direction', [-1, -1, 2]),
                ('system-family', 'A1--Cu--fcc'),
                ('cal

#### 2.8. Modify system

- __sizemults__ list of three sets of two integers specifying how many times the ucell vectors of $a$, $b$ and $c$ are replicated in positive and negative directions when creating system.

- __a_uvw, b_uvw, c_uvw__ specifies how to orient the system by defining the crystal vectors of ucell to place along the three system box vectors.
    
- __atomshift__ is a 3D vector rigid-body shift to apply to atoms in the system. The atomshift vector is relative to the rotated unit cell's box vectors.

- __system__ is an atomman.System to insert the dislocation into. 

- __axes__ specifies the transformation matrix used to rotate the ucell to the system's orientation.

In [10]:
sizemults = '0 1 -40 40 -40 40'

inputs['sizemults'] = sizemults
iprPy.input.interpret('atomman_systemmanipulate', inputs)

system = inputs['initialsystem']
print('# of atoms in system =', system.natoms)

axes = inputs['transformationmatrix']

# of atoms in system = 153600


c:\users\lmh1\documents\python-packages\atomman\atomman\lammps\normalize.py:60: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  transformation = np.linalg.lstsq(vects, system.box.vects)[0].T


### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 disl_relax.template

In [11]:
with open('disl_relax.template', 'w') as f:
    f.write("""#LAMMPS input script for relaxing a dislocation monopole

<atomman_system_info>

<atomman_pair_info>

group move type <group_move>
group hold subtract all move

compute peatom all pe/atom

dump first all custom <maxeval> *.dump id type x y z c_peatom
dump_modify first format <dump_modify_format>
thermo_style custom step pe

fix nomove hold setforce 0.0 0.0 0.0

<anneal_info>

min_modify dmax <dmax>

minimize <etol> <ftol> <maxiter> <maxeval>""")

#### 3.2 dislocationmonopole()

In [12]:
def dislocationmonopole(lammps_command, system, potential, burgers,
                        C, mpi_command=None, axes=None, m=[0,1,0], n=[0,0,1],
                        lineboxvector='a', randomseed=None,
                        etol=0.0, ftol=0.0, maxiter=10000, maxeval=100000,
                        dmax=uc.set_in_units(0.01, 'angstrom'),
                        annealtemp=0.0, bshape='circle',
                        bwidth=uc.set_in_units(10, 'angstrom')):
    """
    Creates and relaxes a dislocation monopole system.
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    system : atomman.System
        The bulk system to add the defect to.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    burgers : list or numpy.array of float
        The burgers vector for the dislocation being added.
    C : atomman.ElasticConstants
        The system's elastic constants.
    mpi_command : str or None, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    axes : numpy.array of float or None, optional
        The 3x3 axes used to rotate the system by during creation.  If given,
        will be used to transform burgers and C from the standard
        crystallographic orientations to the system's Cartesian units.
    randomseed : int or None, optional
        Random number seed used by LAMMPS in creating velocities and with
        the Langevin thermostat.  (Default is None which will select a
        random int between 1 and 900000000.)
    etol : float, optional
        The energy tolerance for the structure minimization. This value is
        unitless. (Default is 0.0).
    ftol : float, optional
        The force tolerance for the structure minimization. This value is in
        units of force. (Default is 0.0).
    maxiter : int, optional
        The maximum number of minimization iterations to use (default is 
        10000).
    maxeval : int, optional
        The maximum number of minimization evaluations to use (default is 
        100000).
    dmax : float, optional
        The maximum distance in length units that any atom is allowed to relax
        in any direction during a single minimization iteration (default is
        0.01 Angstroms).
    annealtemp : float, optional
        The temperature to perform a dynamic relaxation at. (Default is 0.0,
        which will skip the dynamic relaxation.)
    bshape : str, optional
        The shape to make the boundary region.  Options are 'circle' and
        'rect' (default is 'circle').
    bwidth : float, optional
        The minimum thickness of the boundary region (default is 10
        Angstroms).
    
    Returns
    -------
    dict
        Dictionary of results consisting of keys:
        
        - **'dumpfile_base'** (*str*) - The filename of the LAMMPS dump file
          for the relaxed base system.
        - **'symbols_base'** (*list of str*) - The list of element-model
          symbols for the Potential that correspond to the base system's
          atypes.
        - **'Stroh_preln'** (*float*) - The pre-logarithmic factor in the 
          dislocation's self-energy expression.
        - **'Stroh_K_tensor'** (*numpy.array of float*) - The energy
          coefficient tensor based on the dislocation's Stroh solution.
        - **'dumpfile_disl'** (*str*) - The filename of the LAMMPS dump file
          for the relaxed dislocation monopole system.
        - **'symbols_disl'** (*list of str*) - The list of element-model
          symbols for the Potential that correspond to the dislocation
          monopole system's atypes.
        - **'E_total_disl'** (*float*) - The total potential energy of the
          dislocation monopole system.
    """
    # Initialize results dict
    results_dict = {}
    
    # Save initial perfect system
    system.dump('atom_dump', f='base.dump')
    results_dict['dumpfile_base'] = 'base.dump'
    results_dict['symbols_base'] = system.symbols
    
    # Solve Stroh method for dislocation
    stroh = am.defect.Stroh(C, burgers, axes=axes, m=m, n=n)
    results_dict['Stroh_preln'] = stroh.preln
    results_dict['Stroh_K_tensor'] = stroh.K_tensor
    
    # Generate dislocation system by displacing atoms
    disp = stroh.displacement(system.atoms.pos)
    system.atoms.pos += disp
    
    # Apply fixed boundary conditions
    system = disl_boundary_fix(system, bwidth, bshape=bshape, lineboxvector=lineboxvector, m=m, n=n)
    
    # Relax system
    relaxed = disl_relax(lammps_command, system, potential,
                         mpi_command = mpi_command, 
                         annealtemp = annealtemp,
                         etol = etol, 
                         ftol = ftol, 
                         maxiter = maxiter, 
                         maxeval = maxeval)
    
    # Save relaxed dislocation system with original box vects
    system_disl = am.load('atom_dump', relaxed['dumpfile'], symbols=system.symbols)
    
    system_disl.box_set(vects=system.box.vects, origin=system.box.origin)
    system_disl.dump('atom_dump', f='disl.dump')
    results_dict['dumpfile_disl'] = 'disl.dump'
    results_dict['symbols_disl'] = system_disl.symbols
    
    results_dict['E_total_disl'] = relaxed['E_total']
    
    # Cleanup files
    os.remove('0.dump')
    os.remove(relaxed['dumpfile'])
    for dumpjsonfile in glob.iglob('*.dump.json'):
        os.remove(dumpjsonfile)
    
    return results_dict



#### 3.3 disl_boundary_fix()

In [13]:
def disl_boundary_fix(system, bwidth, bshape='circle', lineboxvector='a', m=None, n=None):
    """
    Creates a boundary region by changing atom types.
    
    Parameters
    ----------
    system : atomman.System
        The system to add the boundary region to.
    bwidth : float
        The minimum thickness of the boundary region.
    bshape : str, optional
        The shape to make the boundary region.  Options are 'circle' and
        'box' (default is 'circle').
    lineboxvector : str, optional
        Specifies which of the three box vectors (a, b, or c) the dislocation line is to be
        parallel to.  Default value is 'a'.
    m : array-like object, optional
        Cartesian vector used by the Stroh solution, which is perpendicular to both
        lineboxvector and n.  Default value is [0, 1, 0], which assumes the default value
        for the lineboxvector.
    n : array-like object, optional
        Cartesian vector used by the Stroh solution, which is perpendicular to both
        lineboxvector and m.  Default value is [0, 0, 1], which assumes the default value
        for the lineboxvector.
        
    """
    # Set default values
    if m is None:
        m = np.array([0, 1, 0])
    m = np.asarray(m)
    if n is None:
        n = np.array([0, 0, 1])
    n = np.asarray(n)
    
    # Extract values from system
    natypes = system.natypes
    atype = system.atoms_prop(key='atype')
    pos = system.atoms.pos
    
    # Set line_box_velineboxvectorctor dependent values
    if lineboxvector == 'a':
        u = system.box.avect / system.box.a
        boxvect1 = system.box.bvect
        boxvect2 = system.box.cvect
        pbc = (True, False, False)

    elif lineboxvector == 'b':
        u = system.box.bvect / system.box.b
        boxvect1 = system.box.cvect
        boxvect2 = system.box.avect
        pbc = (False, True, False)

    elif lineboxvector == 'c':
        u = system.box.cvect / system.box.c
        boxvect1 = system.box.avect
        pbc = (False, False, True)

    # Assert u, m, n are all orthogonal
    assert np.isclose(np.dot(u, m), 0.0)
    assert np.isclose(np.dot(u, n), 0.0)
    assert np.isclose(np.dot(m, n), 0.0)
    
    # Get components within mn plane
    mn = np.array([m, n])
    mnvect1 = mn.dot(boxvect1)
    mnvect2 = mn.dot(boxvect2)
    mnorigin = mn.dot(system.box.origin)
    mnpos = mn.dot(pos.T).T
    
    # Compute normal vectors to box vectors
    normal_mnvect1 = np.array([-mnvect1[1], mnvect1[0]])
    normal_mnvect2 = np.array([mnvect2[1], -mnvect2[0]])
    normal_mnvect1 = normal_mnvect1 / np.linalg.norm(normal_mnvect1)
    normal_mnvect2 = normal_mnvect2 / np.linalg.norm(normal_mnvect2)
    
    # Find two opposite boundary corners 
    botcorner = mnorigin
    topcorner = mnorigin + mnvect1 + mnvect2
    
    if bshape == 'box':
        # Project all positions normal to the two mnvectors
        pos_normal_1 = np.dot(mnpos, normal_mnvect1)
        pos_normal_2 = np.dot(mnpos, normal_mnvect2)

        # Adjust atypes of boundary atoms
        atype[(pos_normal_1 < np.dot(botcorner, normal_mnvect1) + bwidth)
             |(pos_normal_2 < np.dot(botcorner, normal_mnvect2) + bwidth)
             |(pos_normal_1 > np.dot(topcorner, normal_mnvect1) - bwidth)
             |(pos_normal_2 > np.dot(topcorner, normal_mnvect2) - bwidth)] += natypes
    
    elif bshape == 'circle':
        def line(p1, p2):
            A = (p1[1] - p2[1])
            B = (p2[0] - p1[0])
            C = (p1[0]*p2[1] - p2[0]*p1[1])
            return A, B, -C

        def intersection(L1, L2):
            D  = L1[0] * L2[1] - L1[1] * L2[0]
            Dx = L1[2] * L2[1] - L1[1] * L2[2]
            Dy = L1[0] * L2[2] - L1[2] * L2[0]
            if D != 0:
                x = Dx / D
                y = Dy / D
                return x,y
            else:
                return False

        # Find two opposite boundary corners 
        botcorner = mnorigin
        topcorner = mnorigin + mnvect1 + mnvect2

        # Define normal lines
        normal_line_1 = line([0,0], normal_mnvect1)
        normal_line_2 = line([0,0], normal_mnvect2)

        # Define boundary lines
        bound_bot1 = line(botcorner, botcorner+mnvect1)
        bound_bot2 = line(botcorner, botcorner+mnvect2)
        bound_top1 = line(topcorner, topcorner+mnvect1)
        bound_top2 = line(topcorner, topcorner+mnvect2)

        # Identify intersection points
        intersections = np.array([intersection(normal_line_1, bound_bot1),
                                  intersection(normal_line_2, bound_bot2),
                                  intersection(normal_line_1, bound_top1),
                                  intersection(normal_line_2, bound_top2)])

        # Compute cylinder radius
        radius = np.min(np.linalg.norm(intersections, axis=1)) - bwidth

        # Adjust atypes of boundary atoms
        atype[np.linalg.norm(mnpos, axis=1) > radius] += natypes
    
    else:
        raise ValueError("Unknown boundary shape type! Enter 'circle' or 'box'")
    
    newsystem = deepcopy(system)
    newsystem.atoms.atype = atype
    newsystem.symbols = list(system.symbols) * 2
    newsystem.pbc = pbc
    newsystem.wrap()

    return newsystem



#### 3.4 disl_relax()

In [14]:
def disl_relax(lammps_command, system, potential,
               mpi_command=None, annealtemp=0.0, randomseed=None,
               etol=0.0, ftol=1e-6, maxiter=10000, maxeval=100000,
               dmax=uc.set_in_units(0.01, 'angstrom')):
    """
    Sets up and runs the disl_relax.in LAMMPS script for relaxing a
    dislocation monopole system.
    
    Parameters
    ----------
    lammps_command :str
        Command for running LAMMPS.
    system : atomman.System
        The system to perform the calculation on.
    potential : atomman.lammps.Potential
        The LAMMPS implemented potential to use.
    mpi_command : str, optional
        The MPI command for running LAMMPS in parallel.  If not given, LAMMPS
        will run serially.
    annealtemp : float, optional
        The temperature to perform a dynamic relaxation at. (Default is 0.0,
        which will skip the dynamic relaxation.)
    randomseed : int or None, optional
        Random number seed used by LAMMPS in creating velocities and with
        the Langevin thermostat.  (Default is None which will select a
        random int between 1 and 900000000.)
    etol : float, optional
        The energy tolerance for the structure minimization. This value is
        unitless. (Default is 0.0).
    ftol : float, optional
        The force tolerance for the structure minimization. This value is in
        units of force. (Default is 0.0).
    maxiter : int, optional
        The maximum number of minimization iterations to use (default is 
        10000).
    maxeval : int, optional
        The maximum number of minimization evaluations to use (default is 
        100000).
    dmax : float, optional
        The maximum distance in length units that any atom is allowed to relax
        in any direction during a single minimization iteration (default is
        0.01 Angstroms).
        
    Returns
    -------
    dict
        Dictionary of results consisting of keys:
        
        - **'logfile'** (*str*) - The name of the LAMMPS log file.
        - **'dumpfile'** (*str*) - The name of the LAMMPS dump file
          for the relaxed system.
        - **'E_total'** (*float*) - The total potential energy for the
          relaxed system.
    """
    
    # Get lammps units
    lammps_units = lmp.style.unit(potential.units)
    
    #Get lammps version date
    lammps_date = lmp.checkversion(lammps_command)['date']
    
    # Define lammps variables
    lammps_variables = {}
    system_info = system.dump('atom_data', f='system.dat',
                              units=potential.units,
                              atom_style=potential.atom_style)
    lammps_variables['atomman_system_info'] = system_info
    lammps_variables['atomman_pair_info'] = potential.pair_info(system.symbols)
    lammps_variables['anneal_info'] = anneal_info(annealtemp, randomseed,
                                                  potential.units)
    lammps_variables['etol'] = etol
    lammps_variables['ftol'] = uc.get_in_units(ftol, lammps_units['force'])
    lammps_variables['maxiter'] = maxiter
    lammps_variables['maxeval'] = maxeval
    lammps_variables['dmax'] = dmax
    lammps_variables['group_move'] = ' '.join(np.array(range(1, system.natypes // 2 + 1), dtype=str))
    
    # Set dump_modify format based on dump_modify_version
    if lammps_date < datetime.date(2016, 8, 3):
        lammps_variables['dump_modify_format'] = '"%d %d %.13e %.13e %.13e %.13e"'
    else:
        lammps_variables['dump_modify_format'] = 'float %.13e'
    
    # Write lammps input script
    template_file = 'disl_relax.template'
    lammps_script = 'disl_relax.in'
    with open(template_file) as f:
        template = f.read()
    with open(lammps_script, 'w') as f:
        f.write(iprPy.tools.filltemplate(template, lammps_variables,
                                         '<', '>'))
    
    # Run LAMMPS
    output = lmp.run(lammps_command, lammps_script, mpi_command)
    thermo = output.simulations[-1]['thermo']
    
    # Extract output values
    results = {}
    results['logfile'] = 'log.lammps'
    results['dumpfile'] = '%i.dump' % thermo.Step.values[-1] 
    results['E_total'] = uc.set_in_units(thermo.PotEng.values[-1],
                                         lammps_units['energy'])
    
    return results



#### 3.5 anneal_info() 

In [15]:
def anneal_info(temperature=0.0, randomseed=None, units='metal'):
    """
    Generates LAMMPS commands for thermo anneal.
    
    Parameters
    ----------
    temperature : float, optional
        The temperature to relax at (default is 0.0).
    randomseed : int or None, optional
        Random number seed used by LAMMPS in creating velocities and with
        the Langevin thermostat.  (Default is None which will select a
        random int between 1 and 900000000.)
    units : str, optional
        The LAMMPS units style to use (default is 'metal').
    
    Returns
    -------
    str
        The generated LAMMPS input lines for performing a dynamic relax.
        Will be '' if temperature==0.0.
    """    
    # Return nothing if temperature is 0.0 (don't do thermo anneal)
    if temperature == 0.0:
        return ''
    
    # Generate velocity, fix nvt, and run LAMMPS command lines
    else:
        if randomseed is None:
            randomseed = random.randint(1, 900000000)
        
        start_temp = 2 * temperature
        tdamp = 100 * lmp.style.timestep(units)
        timestep = lmp.style.timestep(units)
        info = '\n'.join([
            'velocity move create %f %i mom yes rot yes dist gaussian' % (start_temp, randomseed),
            'fix nvt all nvt temp %f %f %f' % (temperature, temperature,
                                               tdamp),
            'timestep %f' % (timestep),
            'thermo 10000',
            'run 10000',
            ])
    
    return info



### 4. Run calculation function(s)

In [16]:
results_dict = dislocationmonopole(lammps_command, system, potential, inputs['burgersvector'], C,
                                   axes = axes,
                                   mpi_command = mpi_command,
                                   m = inputs['stroh_m'],
                                   n = inputs['stroh_n'],
                                   lineboxvector = inputs['dislocation_lineboxvector'],
                                   etol = energytolerance,
                                   ftol = forcetolerance,
                                   maxiter = maxiterations,
                                   maxeval = maxevaluations,
                                   dmax = maxatommotion,
                                   annealtemp = annealtemperature, 
                                   randomseed = randomseed, 
                                   bwidth = inputs['boundarywidth'], 
                                   bshape = boundaryshape)

In [17]:
results_dict.keys()

dict_keys(['dumpfile_base', 'symbols_base', 'Stroh_preln', 'Stroh_K_tensor', 'dumpfile_disl', 'symbols_disl', 'E_total_disl'])

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of length to display results in.
- __energy_unit__ is the unit of energy to display cohesive energies in.
- __energy_per_length_unit__ is the energy per length to report the pre logarithmic energy factor in.

In [18]:
length_unit = 'angstrom'
energy_unit = 'eV'
pressure_unit = 'GPa'
energy_per_length_unit = energy_unit+'/'+length_unit

#### 5.2 Print Stroh method parameters

In [19]:
print('pre-ln factor (alpha) =', uc.get_in_units(results_dict['Stroh_preln'], energy_per_length_unit), energy_per_length_unit)
print('K_tensor =')
print(uc.get_in_units(results_dict['Stroh_K_tensor'], pressure_unit), pressure_unit)

pre-ln factor (alpha) = 0.3811521599172976 eV/angstrom
K_tensor =
[[ 81.07544348   0.          12.76027037]
 [  0.         123.86918865   0.        ]
 [ 12.76027037   0.         127.31323026]] GPa


#### 5.3 List dump files

In [20]:
print('Perfect system is saved as', results_dict['dumpfile_base'])
print('Defect system is saved as',  results_dict['dumpfile_disl'])

Perfect system is saved as base.dump
Defect system is saved as disl.dump
